In [8]:

import TickLoader

import numpy as np
import pandas as pd
import datetime as dt
from statsmodels.tsa.arima_model import ARIMA
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from IPython import get_ipython

In [14]:
sugar_time=['02:30:01','11:59:59']
coffee_time=['03:15:01','12:29:59']
cocoa_time=['03:45:01','10:54:59']

strategy='Cocoa'    # 'Sugar'  'Coffee' 'Cocoa'

if strategy== 'Sugar':
    hrs=sugar_time
elif strategy=='Coffee':
    hrs=coffee_time
else:
    hrs=cocoa_time

start_datetime = '2017-01-16 '+hrs[0]
end_datetime = '2018-02-09 '+hrs[1]


#cocoa_model = pd.read_csv('cocoa_model.csv')
#cocoa_model = cocoa_model.set_index('Pricing')
#ff_cocoa = cocoa_model.axes[0].tolist()
#mod_cocoa = cocoa_model.as_matrix()
# Symbol lists

sym_sugar=['SB  FMH0018!','SB  FMH0018-SB  FMK0018','SB  FMK0018-SB  FMN0018','SB  FMN0018-SB  FMV0018','SB  FMV0018-SB  FMH0019','SB  FMH0019-SB  FMK0019','SB  FMK0019-SB  FMN0019','SB  FMN0019-SB  FMV0019','SB  FMV0019-SB  FMH0020']
sym_coffee=['KC  FMZ0017!','KC  FMZ0017-KC  FMH0018','KC  FMH0018-KC  FMK0018','KC  FMK0018-KC  FMN0018','KC  FMN0018-KC  FMU0018','KC  FMU0018-KC  FMZ0018','KC  FMZ0018-KC  FMH0019']
sym_cocoa_us=['CC  FMH0018!','CC  FMH0018-CC  FMK0018','CC  FMK0018-CC  FMN0018','CC  FMN0018-CC  FMU0018','CC  FMU0018-CC  FMZ0018','CC  FMZ0018-CC  FMH0019']
sym_cocoa_uk=['C   FMH0018!','C   FMH0018-C   FMK0018','C   FMK0018-C   FMN0018','C   FMN0018-C   FMU0018','C   FMU0018-C   FMZ0018','C   FMZ0018-C   FMH0019','6BH8']
sym_cocoa=sym_cocoa_us+sym_cocoa_uk

sym1='CC  FMH0018!'
sym2='C   FMH0018!'
sym3='6BH8'

symbols=[sym_sugar,sym_coffee,sym_cocoa]

# Factor names

ff_sugar=['F1','F2','F3','F4','HKN','bF2','bF3','b_HKN','b_NVH']
ff_coffee=['F1','F2','F3','F4', 'UE','ZHK','UZH']
ff_cocoa=['Arb 3','Arb 5','D1','D2','F3_US','F4_US','F3_UK','F4_UK','ZHK_US','ZHK_UK']

ff_names=[ff_sugar,ff_coffee,ff_cocoa]

# Models

mod_sugar = np.array([[0,.25,.25,.25,.25,0,0,0,0],[0,.375,.125,-.125,-.375,0,0,0,0],
                    [0,.25,-.25,-.25,.25,0,0,0,0],[0,0.125,-.375,.375,-.125,0,0,0,0],
                    [1,-2,1,0,0,0,0,0,0],[0,-1.33333,0,0,2.3333,0,0,0,-1],[0,0,0,0,.25,0,-0.5,0,0.25],
                    [0,0,0,0,1,-2,1,0,0],[0,0,0,0,0,0,1,-2,1]])
                  
              
mod_coffee = np.array([[0,0.25,0.25,0.25,0,0.25,0],[0,0.34091,0.1591,-0.022727,0,-0.477273,0],
                     [0,0.3099795,-0.1449077,-0.3550923,0,0.1900205,0],
                     [0,0.1785714,-0.459184,0.3214826,0,-0.0408163,0],
                     [0,-0.33333,1.2857143,-1.8,1,-0.152381,0],
                     [1,-2.5,1.5,0,0,0,0],[0,0,0,0.6,0,-1.6,1]])

mod_cocoa = np.array([[0,0,1,0,0,0,0,0,-1,0,0],[0,0,0,0,1,0,0,0,0,-1],
                     [0,0.25,0.25,.25,0.25,0,-.25,-.25,-.25,-.25],[0,0.375,0.125,-0.125,-0.375,0,-0.375,-0.125,0.125,0.375],
                     [0,0.25,-0.25,-0.25,0.25,0,0,0,0,0],[0,0.125,-0.375,0.375,-0.125,0,0,0,0,0],
                     [0,0,0,0,0,0,0.25,-0.25,-0.25,0.25],[0,0,0,0,0,0,0.125,-0.375,0.375,-0.125],
                     [1,-1.075,-.525,0.025,0.575,0,-.825,-.275,.275,0.825],[0,-0.4125,-0.1375,0.1375,0.4125,1,-0.6625,-0.3875,-0.1125,0.1625]])

models=[mod_sugar,mod_coffee,mod_cocoa]

# EMA length

tau_sugar=[1,1,99999,99999,300,1,99999,99999,99999]
tau_coffee=[1,1,99999,99999,99999,600,900]
tau_cocoa=[1,1,180,60,99999,99999,99999,99999,30,90,360,180]

taus=[tau_sugar,tau_coffee,tau_cocoa]
taus=[2*np.asarray(tau) for tau in taus] #adjustmnet for panda ema fornula

In [15]:

taus=[tau_sugar,tau_coffee,tau_cocoa]
taus=[2*np.asarray(tau) for tau in taus] #adjustmnet for panda ema fornula

# Market hours



# Input dataframe - all strategies

inputs=pd.DataFrame(index=['Symbols','Factors','Models','EMA_Length'],
                columns=['Sugar','Coffee','Cocoa'])

inputs.loc['Symbols']=symbols
inputs.loc['Factors']=ff_names
inputs.loc['Models']=models
inputs.loc['EMA_Length']=taus

# Data load and clean up

specs=inputs[strategy]
syms=specs[0]
fac_names=specs[1]
model=specs[2]
ema_len=specs[3]

def filter_trading_hours(tmp_series):
    tmp_series = tmp_series.drop(tmp_series[tmp_series.index.weekday == 5].index)
    tmp_series = tmp_series.drop(tmp_series[tmp_series.index.weekday == 6].index)
    return tmp_series


tick_list = []
for sym in syms:
    tick_list  += [TickLoader.resample_ticks(sym, start_datetime, end_datetime,
                                              frequency='1T', use_weighted_mids=False)]

raw_ticks = pd.concat(tick_list, axis=1)
filtered_ticks = filter_trading_hours(raw_ticks)

# Outright series generation

if strategy=='Cocoa':
    gbp=filtered_ticks[syms[len(syms)-1]]/10000
    syms=[x for x in syms[0:len(syms)-1]]
    
    syms2=[x for x in syms if x[0:2]==syms[len(syms)-1][0:2]]
    syms=[x for x in syms if x not in syms2]
    contracts2=[sym.split("-")[1] for sym in syms2[1:len(syms2)]]
    contracts2.insert(0,syms2[0])
    outrights2=pd.concat([filtered_ticks[syms2[0]],
                     pd.concat([filtered_ticks[syms2[0]]-filtered_ticks.iloc[:,len(syms)+1:].cumsum(axis=1)[sym] 
                     for sym in syms2[1:len(syms2)]],axis=1)],axis=1)
    outrights2.columns=contracts2
    outrights2=pd.concat([outrights2[sym]*gbp for sym in contracts2],axis=1)

contracts=[sym.split("-")[1] for sym in syms[1:len(syms)]]
contracts.insert(0,syms[0])

outrights=pd.concat([filtered_ticks[syms[0]],
                     pd.concat([filtered_ticks[syms[0]]-filtered_ticks.iloc[:,1:].cumsum(axis=1)[sym] 
                     for sym in syms[1:len(syms)]],axis=1)],axis=1)
outrights.columns=contracts

if strategy=='Cocoa':
    outrights=pd.concat([outrights,outrights2],axis=1)


# Factor Prices, EMAs, Forecasts

factors_day=outrights.dot(model.T)
factors_day.columns=fac_names
factors_day=factors_day.dropna()

ema_day = pd.concat([pd.ewma(factors_day[fac_name], span=tau) 
                    for fac_name, tau in zip(fac_names, ema_len)],
                    axis=1)

test = factors_day.sub(ema_day)

Symbol/ID: <CC  FMH0018!/376734465458809590> was not found in tickdb folder: </media/tickdb/TickFS//ICE> on days: <2017-01-16, 2018-02-09>. This symbol will be removed from calls to TickP.


IndexError: index 0 is out of bounds for axis 0 with size 0

In [5]:
# Charts

get_ipython().run_line_magic('matplotlib', 'inline')

b=np.empty(1500)
b[:]=np.nan
t=range(1,len(factors_day.index)+1)

fig = plt.figure(1,figsize=(10, 13))

inner = gridspec.GridSpec(3, 2, wspace=0.2, hspace=0.2)
for i in range(2):
    f_i=fac_names[i]
    ax = plt.Subplot(fig, inner[i],title=f_i)
    ax.set_color_cycle(['darkslategray', 'mediumaquamarine'])
    ax.plot(t,factors_day[f_i].values,t,ema_day[f_i].values)
    fig.add_subplot(ax)
    plt.grid()
for i in range(2,min(len(fac_names),6)):
    f_i=fac_names[i]
    a=np.concatenate((ema_day[f_i].values,b),axis=0)
    c=np.concatenate((factors_day[f_i].values,b),axis=0)
    ax = plt.Subplot(fig, inner[i],title=f_i)
    ax.set_color_cycle(['seagreen', 'darkslategray', 'mediumaquamarine'])
    ax.plot(c)
    ax.plot(a)
    fig.add_subplot(ax)
    plt.grid()    
fig.show()

if len(fac_names)>6:
    fig = plt.figure(3,figsize=(10, 13))
    inner = gridspec.GridSpec(3, 2, wspace=0.2, hspace=0.2)
    for i in range(len(fac_names)-6):
        f_i=fac_names[i+6]
        a=np.concatenate((ema_day[f_i].values,b),axis=0)
        c=np.concatenate((factors_day[f_i].values,b),axis=0)
        ax = plt.Subplot(fig, inner[i],title=f_i)
        ax.set_color_cycle(['seagreen', 'darkslategray', 'mediumaquamarine'])
        ax.plot(c)
        ax.plot(a)
        fig.add_subplot(ax)
        plt.grid()     
fig.show()


NameError: name 'factors_day' is not defined

In [ ]:
test = factors_day.sub(ema_day)

In [ ]:
plt.plot(test['d2'].values)
plt.title('arb minus ema for 3 hours')
plt.show()